In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/drive/MyDrive/stores_tables_us.xlsx')

In [ ]:
df.head()

,Unnamed: 0,StoreID,City,State,ZipCode,Longitude,Latitude
0,0,8382,Yuma,AZ,85365.0,-114.608225,32.669012
1,1,8383,Yuma,AZ,85365.0,-114.584900,32.601897
2,2,8384,Yuma,AZ,85367.0,-114.443464,32.658674
3,3,8388,Surprise,AZ,85378.0,-112.348586,33.639345
4,4,8390,Peoria,AZ,85382.0,-112.247623,33.638848


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50824 entries, 0 to 50823
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  50824 non-null  int64  
 1   StoreID     50824 non-null  int64  
 2   City        50824 non-null  object 
 3   State       50824 non-null  object 
 4   ZipCode     50823 non-null  float64
 5   Longitude   50824 non-null  float64
 6   Latitude    50824 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 2.7+ MB


In [ ]:
import io

df1 = df.sample(n=500, random_state=42)
df1.dropna(subset=["Latitude", "Longitude"], inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster

In [ ]:
m = folium.Map(location=[37.09, -95.71], zoom_start=4)

In [ ]:
folium.plugins.Draw(export=True, filename='data.geojson', position='topleft', draw_options=None, edit_options=None).add_to(m)
m

In [ ]:
m = folium.Map(location=[37.09, -95.71], zoom_start=4)

locations = list(zip(df1.Latitude, df1.Longitude))
icons = [folium.Icon(icon="store", prefix="fa") for _ in range(len(locations))]

cluster = MarkerCluster(locations=locations, icons=icons)
popup1 = folium.LatLngPopup()

folium.plugins.Draw(export=True, filename='data.geojson', position='topleft', draw_options=None, edit_options=None).add_to(m)

m.add_child(popup1)
m.add_child(cluster)
m

In [ ]:
def get_geojson_grid(upper_right, lower_left, n=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [ ]:
lower_left = [26.3963, -125.2437]
upper_right = [49.2538, -72.3147]
m = folium.Map(location=[37.09, -95.71], zoom_start=4)
grid = get_geojson_grid(upper_right, lower_left , n=6)

for i, geo_json in enumerate(grid):

    color = plt.cm.Reds(i / len(grid))
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })
    popup = folium.Popup("example popup {}".format(i))
    gj.add_child(popup)

    m.add_child(gj)
m

In [ ]:
import json


# Generate GeoJson grid
lower_left = [26.3963, -125.2437]
upper_right = [49.2538, -72.3147]

m = folium.Map(location=[37.09, -95.71], zoom_start=4)

# Calculate exposures in grid
popups = []
regional_counts = []

for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]

    mask = ((df1.Latitude<=upper_right[1]) & (df1.Latitude<=lower_left[1]) & (df1.Longitude<=upper_right[0]) & (df1.Longitude<=lower_left[0]))

    region_incidents = len(df1[mask])
    regional_counts.append(region_incidents)

    total_stores = df1[mask].StoreID.count()
    content = "total no.of stores {:,.0f}".format(total_stores)
    popup = folium.Popup(content)
    popups.append(popup)

worst_region = max(regional_counts)

# Add GeoJson to map
for i, box in enumerate(grid):
    geo_json = json.dumps(box)

    color = plt.cm.Reds(regional_counts[i] / worst_region)
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.55,
                                                                    })

    gj.add_child(popups[i])
    m.add_child(gj)

locations = list(zip(df1.Latitude, df1.Longitude))
icons = [folium.Icon(icon="store", prefix="fa") for _ in range(len(locations))]
folium.plugins.Draw(export=True, filename='data.geojson', position='topleft', draw_options=None, edit_options=None).add_to(m)
popup1 = folium.LatLngPopup()

cluster = MarkerCluster(locations=locations, icons=icons)
m.add_child(cluster)
m.add_child(popup1)